In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

In [4]:
model_name = "distilbert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

c:\Users\letic\OneDrive\Área de Trabalho\Git repos\llm-lora-finetuning\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\letic\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForQuestionAnswering were not initializ

# Pre-processing data

In [13]:
schopenhauer_df = pd.read_csv('data/gpt-schopenn.csv').rename(columns={'Quotes':'output_text', 'Question':'input_text'})
schopenhauer_df.head()

,output_text,input_text
0,It is difficult to find happiness within onese...,Happiness and Inner Self
1,"All truth passes through three stages. First, ...",The Nature of Truth
2,"The wise have always said the same things, and...",Wisdom vs. Foolishness
3,The majority of men are not capable of thinkin...,Critical Thinking vs. Blind Belief
4,A pessimist is an optimist in full possession ...,Pessimism and Realism


In [21]:

INSTRUCTION_TEMPLATE = f"""\
Please answer the questions about the following subjects as the philosopher Arthur Schopenhauer. \
Answer it with a quote from you about the subject.\
\
Student question: \
Please share your knowledge about 
"""


In [22]:
schopenhauer_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' ' + schopenhauer_df['input_text']

In [23]:
schopenhauer_df.head()

,output_text,input_text,input_text_instruct
0,It is difficult to find happiness within onese...,Happiness and Inner Self,Please answer the questions about the followin...
1,"All truth passes through three stages. First, ...",The Nature of Truth,Please answer the questions about the followin...
2,"The wise have always said the same things, and...",Wisdom vs. Foolishness,Please answer the questions about the followin...
3,The majority of men are not capable of thinkin...,Critical Thinking vs. Blind Belief,Please answer the questions about the followin...
4,A pessimist is an optimist in full possession ...,Pessimism and Realism,Please answer the questions about the followin...


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)